<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_advanced_universal_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch matplotlib scikit-learn umap-learn

In [ ]:
#!/usr/bin/env python3
# train_advanced_universal_ai.py

import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils import clip_grad_norm_
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
try:
    import umap
    USE_UMAP = True
except ImportError:
    USE_UMAP = False

# 1. Reproducibility & Device
SEED   = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Hyperparameters
INPUT_DIM        = 6
HIDDEN_DIM       = 32
OUTPUT_DIM       = 3
LR               = 1e-3
BATCH_SIZE       = 64
EPOCHS           = 100
DATASET_SZ       = 2000
DROPOUT_P        = 0.2
CLIP_NORM        = 1.0
PHY_LOSS_LAMBDA  = 1.0

# 3. Synthetic Data Generators
def make_synthetic_dataset(seed_offset=0):
    torch.manual_seed(SEED + seed_offset)
    X = torch.randn(DATASET_SZ, INPUT_DIM)
    W_true = torch.randn(INPUT_DIM, OUTPUT_DIM)
    y = X @ W_true + 0.05 * torch.randn(DATASET_SZ, OUTPUT_DIM)
    return TensorDataset(X, y)

dissolution_ds = make_synthetic_dataset(0)
precomp_ds     = make_synthetic_dataset(1)
diss_loader    = DataLoader(dissolution_ds, batch_size=BATCH_SIZE, shuffle=True)
precomp_loader = DataLoader(precomp_ds, batch_size=BATCH_SIZE, shuffle=True)

# 4. Base Model with MC-Dropout
class BaseAI(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1     = nn.Linear(INPUT_DIM, HIDDEN_DIM)
        self.dropout = nn.Dropout(DROPOUT_P)
        self.relu    = nn.ReLU()
        self.fc2     = nn.Linear(HIDDEN_DIM, OUTPUT_DIM)

    def forward(self, x):
        h = self.fc1(x)
        h = self.dropout(h)
        h = self.relu(h)
        return self.fc2(h)

class DissolutionAI(BaseAI):       pass
class PreComputationalAI(BaseAI):  pass

# 5. Physics-Informed Loss
def physics_loss(outputs):
    # penalize negative value in index=2 (“field stability”)
    negative_part = torch.relu(-outputs[:, 2])
    return PHY_LOSS_LAMBDA * negative_part.mean()

# 6. Training Routine
def train_model(model, loader, name):
    model.to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.5, patience=5
    )
    mse_loss = nn.MSELoss()
    history  = []

    for epoch in range(1, EPOCHS + 1):
        model.train()
        epoch_loss = 0.0

        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)

            optimizer.zero_grad()
            preds = model(xb)
            loss  = mse_loss(preds, yb) + physics_loss(preds)
            loss.backward()
            clip_grad_norm_(model.parameters(), CLIP_NORM)
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(loader)
        history.append(avg_loss)
        scheduler.step(avg_loss)

        if epoch % 10 == 0 or epoch == 1:
            print(f"[{name}] Epoch {epoch:3d}/{EPOCHS} — Loss: {avg_loss:.6f}")

    # Plot training curve
    plt.figure(figsize=(5,3))
    plt.plot(history, label=f"{name} Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"{name} Training")
    plt.legend()
    plt.tight_layout()
    plt.show()

    return model

# 7. Latent-Space Visualization
def visualize_latent_space(model, dataset, title):
    X_all = torch.stack([x for x, _ in dataset]).to(DEVICE)
    with torch.no_grad():
        h = model.fc1(X_all).cpu().numpy()  # pre-dropout activations

    if USE_UMAP:
        reducer = umap.UMAP(n_components=2, random_state=SEED)
    else:
        reducer = TSNE(n_components=2, random_state=SEED)

    h2d = reducer.fit_transform(h)
    plt.figure(figsize=(5,5))
    plt.scatter(h2d[:,0], h2d[:,1], s=5, alpha=0.6)
    plt.title(f"{title} Latent Space ({'UMAP' if USE_UMAP else 't-SNE'})")
    plt.tight_layout()
    plt.show()

# 8. MC-Dropout Inference (fixed detach)
def mc_dropout_inference(model, x, samples=50):
    model.train()
    preds = []
    for _ in range(samples):
        out = model(x.to(DEVICE))
        preds.append(out.detach().cpu().numpy())
    arr = np.stack(preds, axis=0)
    return arr.mean(axis=0), arr.std(axis=0)

# 9. Main Execution
if __name__ == "__main__":
    # Train both models
    dissolution_model = train_model(DissolutionAI(), diss_loader,    "DissolutionAI")
    precomp_model     = train_model(PreComputationalAI(), precomp_loader, "PreComputationalAI")

    # Visualize latent spaces
    visualize_latent_space(dissolution_model, dissolution_ds,    "DissolutionAI")
    visualize_latent_space(precomp_model,     precomp_ds,          "PreComputationalAI")

    # Sample inference + uncertainty
    x_sample = torch.randn(1, INPUT_DIM)
    mean_d, std_d = mc_dropout_inference(dissolution_model, x_sample)
    mean_p, std_p = mc_dropout_inference(precomp_model,     x_sample)

    print("\nDissolutionAI Output Mean :", mean_d.flatten())
    print("DissolutionAI Output Std  :", std_d.flatten())
    print("\nPreComputationalAI Output Mean:", mean_p.flatten())
    print("PreComputationalAI Output Std :", std_p.flatten())